In [0]:
import tensorflow as tf
print(tf.__version__)

tf.enable_eager_execution()

1.14.0


In [0]:
import tensorflow_datasets as tfds
imdb, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True)

Dataset is loaded. Let us now convert the sentences into tokenized sequences.

In [0]:
import numpy as np

train_data, test_data = imdb['train'], imdb['test']

training_sentences = []
training_labels = []

testing_sentences = []
testing_labels = []

# str(s.tonumpy()) is needed in Python3 instead of just s.numpy()
for s,l in train_data:
  training_sentences.append(str(s.numpy()))
  training_labels.append(l.numpy())
  
for s,l in test_data:
  testing_sentences.append(str(s.numpy()))
  testing_labels.append(l.numpy())
  
training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)


In [0]:
vocab_size = 10000
embedding_dim = 16
max_length = 120
trunc_type='post'
oov_tok = "<OOV>"


from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences,maxlen=max_length, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences,maxlen=max_length)



In [0]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

print(decode_review(padded[1]))
print(training_sentences[1])

b'i have no idea what the other reviewer is talking about this was a wonderful movie and created a sense of the era that feels like time travel the characters are truly young mary is a strong match for byron claire is juvenile and a tad annoying <OOV> is a convincing beaten down <OOV> all are beautiful curious and <OOV> not the frightening <OOV> they are in gothic br br gothic works as an independent piece of shock film and i loved it for different reasons but this works like a <OOV> and <OOV> film and was from my <OOV> the best capture of what the summer must have felt like romantic yes but completely <OOV> my interest in the
b'I have no idea what the other reviewer is talking about- this was a wonderful movie, and created a sense of the era that feels like time travel. The characters are truly young, Mary is a strong match for Byron, Claire is juvenile and a tad annoying, Polidori is a convincing beaten-down sycophant... all are beautiful, curious, and decadent... not the frightening

In [0]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 120, 16)           160000    
_________________________________________________________________
flatten (Flatten)            (None, 1920)              0         
_________________________________________________________________
dense (Dense)                (None, 6)                 11526     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 7         
Total params: 171,533
Trainable params: 171,533
Non-trainable params: 0
_________________________________________________________________


In [0]:
num_epochs = 10
model.fit(padded, training_labels_final, epochs=num_epochs, validation_data=(testing_padded, testing_labels_final))

W0707 10:40:33.592673 140656975026048 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 25000 samples, validate on 25000 samples
Epoch 1/10
25000/25000 [==============================] - 6s 256us/sample - loss: 0.4975 - acc: 0.7366 - val_loss: 0.3494 - val_acc: 0.8486
Epoch 2/10
25000/25000 [==============================] - 5s 217us/sample - loss: 0.2402 - acc: 0.9080 - val_loss: 0.3736 - val_acc: 0.8354
Epoch 3/10
25000/25000 [==============================] - 5s 216us/sample - loss: 0.0941 - acc: 0.9764 - val_loss: 0.4504 - val_acc: 0.8275
Epoch 4/10
25000/25000 [==============================] - 5s 217us/sample - loss: 0.0246 - acc: 0.9969 - val_loss: 0.5303 - val_acc: 0.8234
Epoch 5/10
25000/25000 [==============================] - 5s 217us/sample - loss: 0.0055 - acc: 0.9997 - val_loss: 0.6038 - val_acc: 0.8243
Epoch 6/10
25000/25000 [==============================] - 5s 216us/sample - loss: 0.0018 - acc: 1.0000 - val_loss: 0.6484 - val_acc: 0.8258
Epoch 7/10
25000/25000 [==============================] - 5s 216us/sample - loss: 8.6255e-04 - acc: 1.0000 - v

In [0]:
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape) # shape: (vocab_size, embedding_dim)

(10000, 16)


In [0]:
import io

out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')
for word_num in range(1, vocab_size):
  word = reverse_word_index[word_num]
  embeddings = weights[word_num]
  out_m.write(word + "\n")
  out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
out_v.close()
out_m.close()

In [0]:


try:
  from google.colab import files
except ImportError:
  pass
else:
  files.download('vecs.tsv')
  files.download('meta.tsv')

Now we can visualise these embeddings on [Embedding Projector](https://projector.tensorflow.org/).